
- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [225]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags

In [226]:
# load one file first to see what type of data you're dealing with and what attributes it has

directory = '../data/'

with open(f'{directory}AK_Juneau_0.json') as f:
    jsonfile = json.load(f)

testcity = pd.json_normalize(jsonfile['data']['results']).head(3)

jsoncolumns = testcity.columns

testcity

,last_update_date,tags,permalink,status,list_date,open_houses,branding,list_price,property_id,photos,...,location.address.state_code,location.address.line,location.street_view_url,location.county.fips_code,location.county.name,primary_photo,source,products,location.address.coordinate,other_listings
0,2023-09-19T20:52:50Z,"[carport, community_outdoor_space, cul_de_sac,...",9453-Herbert-Pl_Juneau_AK_99801_M90744-30767,sold,2023-06-29T21:16:25.000000Z,None,"[{'name': 'EXP Realty LLC - Southeast Alaska',...",554950.0,9074430767,"[{'tags': [{'label': 'house_view', 'probabilit...",...,AK,9453 Herbert Pl,https://maps.googleapis.com/maps/api/streetvie...,None,Juneau,NaN,NaN,NaN,NaN,NaN
1,None,None,8477-Thunder-Mountain-Rd_Juneau_AK_99801_M9424...,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,9424983842,None,...,AK,8477 Thunder Mountain Rd,https://maps.googleapis.com/maps/api/streetvie...,None,Juneau,NaN,NaN,NaN,NaN,NaN
2,None,None,4515-Glacier-Hwy_Juneau_AK_99801_M94790-68516,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,9479068516,None,...,AK,4515 Glacier Hwy,https://maps.googleapis.com/maps/api/streetvie...,None,Juneau,NaN,NaN,NaN,NaN,NaN


In [227]:
import warnings

df = pd.DataFrame(columns= jsoncolumns)

json_files = [file for file in os.listdir(directory) if file.endswith('.json')]

for file in json_files:
    file_path = os.path.join(directory, file)
    json_subfile = pd.read_json(file_path)
    json_subdf = pd.json_normalize(json_subfile['data']['results'])
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        df = pd.concat([df, json_subdf], ignore_index=True)
    
df.shape

(8191, 67)

In [228]:
[col for col in df.columns if col not in jsoncolumns]

['community.advertisers', 'community.description.name', 'location.county']

At this point, ensure that you have all sales in a dataframe.
- Is each cell one value, or do some cells have lists?
- Maybe the "tags" will help create some features.
- What are the data types of each column?
- Some sales may not actually include the sale price.  These rows should be dropped.
- Some sales don't include the property type.
- There are a lot of None values.  Should these be dropped or replaced with something?

In [229]:
df.shape

(8191, 67)

In [230]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8191 entries, 0 to 8190
Data columns (total 67 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   last_update_date                       8125 non-null   object 
 1   tags                                   7638 non-null   object 
 2   permalink                              8159 non-null   object 
 3   status                                 8159 non-null   object 
 4   list_date                              7752 non-null   object 
 5   open_houses                            0 non-null      object 
 6   branding                               8159 non-null   object 
 7   list_price                             7721 non-null   object 
 8   property_id                            8159 non-null   object 
 9   photos                                 7403 non-null   object 
 10  community                              0 non-null      object 
 11  virt

In [231]:
df.describe()

,price_reduced_amount,description.year_built,description.baths_3qtr,description.baths_full,description.baths_half,description.lot_sqft,description.sqft,description.baths,description.garage,description.stories,description.beds,location.address.coordinate.lon,location.address.coordinate.lat,location.county
count,2.484000e+03,7316.000000,566.000000,7311.000000,2281.000000,6.991000e+03,7323.000000,7980.000000,4448.000000,6260.000000,7504.000000,7909.000000,7909.000000,0.0
mean,2.442704e+04,1968.916074,1.247350,1.887430,1.105655,2.510949e+05,1933.848559,2.131203,1.926709,1.567732,3.208289,-92.206522,39.009689,NaN
std,7.162396e+04,35.096914,0.463482,0.862214,0.413340,5.823820e+06,1339.039206,1.175940,0.878766,0.730969,1.282732,15.888886,4.374553,NaN
min,1.000000e+02,1828.000000,1.000000,1.000000,1.000000,0.000000e+00,120.000000,0.000000,1.000000,1.000000,0.000000,-157.810583,21.277707,NaN
25%,6.000000e+03,1950.000000,1.000000,1.000000,1.000000,4.953000e+03,1258.000000,1.000000,1.000000,1.000000,3.000000,-104.971611,35.688084,NaN
50%,1.010000e+04,1975.000000,1.000000,2.000000,1.000000,7.841000e+03,1635.000000,2.000000,2.000000,1.000000,3.000000,-89.333131,39.698210,NaN
75%,2.000000e+04,1997.000000,1.000000,2.000000,1.000000,1.263200e+04,2264.000000,3.000000,2.000000,2.000000,4.000000,-78.617690,41.832266,NaN
max,2.015999e+06,2024.000000,3.000000,8.000000,5.000000,1.676624e+08,32106.000000,9.000000,11.000000,10.000000,12.000000,-71.006343,58.396178,NaN


In [232]:
# load and concatenate data here
# drop or replace values as necessary

# print(df.iloc[0,0:10])
# print(df.iloc[0,10:20])
# print(df.iloc[0,20:30])



In [233]:
# print(df.iloc[0,30:40])
# print(df.iloc[0,40:50])
# print(df.iloc[0,50:60])
# print(df.iloc[0,60:70])

In [234]:
print(df['tags'][0])

['central_air', 'dishwasher', 'fireplace', 'forced_air', 'hardwood_floors', 'washer_dryer', 'basement', 'single_story', 'garage_1_or_more', 'garage_2_or_more', 'big_yard', 'vaulted_ceiling', 'open_floor_plan', 'floor_plan', 'corner_lot', 'big_lot', 'ensuite']


In [235]:
print(df['branding'][0])

[{'name': 'Howard Hanna - Colonial Park', 'photo': None, 'type': 'Office'}]


In [236]:
print(df['photos'][0]) 

[{'tags': [{'label': 'garage', 'probability': 0.99981635808945}, {'label': 'house_view', 'probability': 0.99869686365128}, {'label': 'road_view', 'probability': 0.99956554174423}, {'label': 'yard', 'probability': 0.9751615524292}, {'label': 'house_view', 'probability': 0.99909782409668}, {'label': 'house_view', 'probability': 0.99994349479675}], 'href': 'https://ap.rdcpix.com/8c0b391ae4f9bbca36c41d559ef75598l-m2349710986s-w1024_h768.jpg'}]


In [237]:
# [value for value in df['price_reduced_amount'] if value is not np.NaN]


In [238]:

drop_cols = ['last_update_date', 'permalink', 'list_date', 'open_houses', 'branding', 
             'property_id', 'photos', 'community', 'virtual_tours', 'listing_id', 'matterport',
             'primary_photo.href', 'source.plan_id', 'source.agents', 'source.spec_id', 
             'source.type', 'description.name','lead_attributes.show_contact_an_agent',
               'price_reduced_amount', 'flags.is_for_rent', 'flags.is_contingent',
               'flags.is_pending', 'flags.is_plan', 'flags.is_coming_soon', 'flags.is_new_listing',
               'products.brand_name', 'other_listings.rdc', 'location.address.postal_code', 'location.address.line',
               'location.street_view_url', 'location.county.fips_code', 'primary_photo', 'source', 'products',
               'location.address.coordinate', 'other_listings', 'community.advertisers', 'community.description.name',
               'location.county', 'status', 'location.address.state', 'flags.is_new_construction', 
               'flags.is_subdivision', 'flags.is_price_reduced', 'flags.is_foreclosure']

            

df = df.drop(columns= drop_cols)

In [239]:
df.shape

(8191, 22)

In [240]:
# df.iloc[0:5, :10]

In [241]:
# df.iloc[0:5,10:]

In [242]:
# df['flags.is_price_reduced'].value_counts()/df.shape[0]

In [243]:
# df['flags.is_foreclosure'].value_counts()/df.shape[0]

In [244]:
missing_tags = df[df['tags'].isna()]
missing_tags.shape[0]/df.shape[0]

0.06751312416066414

In [245]:
# Extract tags
tags = list({tag for sublist in df['tags'] if isinstance(sublist, list) for tag in sublist})

# Dataframe for tags
tag_df = pd.DataFrame(0, index=df.index, columns=tags)

# Fill in tags
for tag in tags:
    tag_df[tag] = df['tags'].apply(lambda x: 1 if isinstance(x, list) and tag in x else 0)

# Concatenate tag df and original feature set
df = pd.concat([df, tag_df], axis=1)

# Drop the original 'tags' column
df.drop('tags', axis=1, inplace=True)


In [246]:
df = df[~df['description.sold_price'].isna()] # drop missing sell prices

In [247]:
df = df[~df['description.type'].isna()] # drop missing property type
df.shape

(6696, 176)

In [248]:
# original_features = df.iloc[:,0:21] #full feature set is too big for info()
# original_features.info()

In [249]:
df = df.drop(columns=['list_price', 'description.sold_date', 'description.baths_1qtr'])
df = df[~df['description.year_built'].isna()]
df = df[~df['description.lot_sqft'].isna()]
df = df[~df['description.sqft'].isna()]
df = df[~df['location.address.coordinate.lon'].isna()]
df = df[~df['location.address.coordinate.lat'].isna()]

columns_to_fill_zeroes = ['description.baths_3qtr', 'description.baths_full', 'description.baths_half', 'description.baths',
                          'description.garage', 'description.beds']
df[columns_to_fill_zeroes] = df[columns_to_fill_zeroes].fillna(0)

columns_to_fill_NA = ['description.sub_type', 'location.address.city', 'location.county.name']
df[columns_to_fill_NA] = df[columns_to_fill_NA].fillna('N/A')

columns_to_fill_ones = ['description.stories']
df[columns_to_fill_ones] = df[columns_to_fill_ones].fillna(1)


In [250]:
original_features = df.iloc[:,0:18] 
original_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5260 entries, 0 to 8190
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   description.year_built           5260 non-null   float64
 1   description.baths_3qtr           5260 non-null   float64
 2   description.sold_price           5260 non-null   object 
 3   description.baths_full           5260 non-null   float64
 4   description.baths_half           5260 non-null   float64
 5   description.lot_sqft             5260 non-null   float64
 6   description.sqft                 5260 non-null   float64
 7   description.baths                5260 non-null   float64
 8   description.sub_type             5260 non-null   object 
 9   description.garage               5260 non-null   float64
 10  description.stories              5260 non-null   float64
 11  description.beds                 5260 non-null   float64
 12  description.type    

In [251]:
df.shape

(5260, 173)

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.
- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE such as using central tendency?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.

In [252]:
# OHE categorical variables here
# tags will have to be done manually

If using numerical values to replace cities or states, make sure that the data is split so that we don't leak data into the training selection.
- If you think about a real scenario, do we have any knowledge of new listings that would come in once modeling is complete?
- Drop columns that aren't needed.
- Don't keep the list price because it will be too close to the sale price.

In [253]:
# perform train test split here
# do something with state and city

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [254]:
# perform EDA here

Now is a great time to scale the data and save it once it's preprocessed.